# Table of Contents
- [Import Packages](#Packages)
- [Data Cleaning](#Data-Cleaning)
- [Data Exploration](#Data-Exploration)
- [Models](#Models)


## Packages

In [27]:
import os
import time
import csv
import subprocess
import psutil
from datetime import datetime
from fuzzywuzzy import fuzz,process
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
from feature_engine.imputation import CategoricalImputer

from boruta import BorutaPy

from fancyimpute import KNN
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score,confusion_matrix, ConfusionMatrixDisplay,roc_curve,r2_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


import category_encoders as ce
import zipfile
import sweetviz as sv

import xgboost as xgb
import optuna
import optuna.visualization as vis

import matplotlib.pyplot as plt
edu_mapping = {
    "00": 0,   # NIU (Not in Universe)
    "01": 0,   # No school completed
    "02": 0,   # NIU or blank
    "03": 2,   # 1st-4th grade
    "04": 4,   # 5th-8th grade
    "05": 8,   # 9th grade
    "06": 9,   # 10th grade
    "07": 10,  # 11th grade
    "08": 11,  # 12th grade, no diploma
    "09": 12,  # High school graduate, GED
    "10": 13,  # Some college, no degree
    "11": 14,  # Associate degree, type of program not specified
    "12": 14,  # Associate degree, occupational program
    "13": 14,  # Associate degree, academic program
    "14": 16,  # Bachelor's degree
    "15": 18,  # Master's degree
    "16": 19,  # Professional degree
    "17": 20   # Doctorate degree
}

def recode_educ_column(series: pd.Series) -> pd.Series:
    """Convert to string, zero-fill to length 3, then map with the dictionary."""
    return (
        series.astype(str)
              .str.zfill(3)
              .map(edu_mapping)
              .astype(float)  # Ensures NaNs are properly handled
    )


age_bins = [0, 18, 25, 35, 45, 55, 65, np.inf]
age_labels = ["under_18", "18–24", "25–34", "35–44", "45–54", "55–64", "65+"]

def pick_two_parents(row):
    # Gather all four parent columns into a list
    ages = [row["AGE_MOM"], row["AGE_MOM2"], row["AGE_POP"], row["AGE_POP2"]]
    valid = [age for age in ages if pd.notna(age)]
    
    parent1 = valid[0] if len(valid) > 0 else np.nan
    parent2 = valid[1] if len(valid) > 1 else np.nan
    
    return pd.Series([parent1, parent2], index=["parent1_age", "parent2_age"])

# Apply the function row-by-row



## Data-Cleaning

In [34]:
df = pd.read_csv("cps_00018.csv",nrows = 100000)
columns_all_nan = [col for col in df.columns if df[col].isna().all()]
print(f"Columns entirely NaN: {columns_all_nan}")

# Drop columns that are entirely NaN
df = df.drop(columns=columns_all_nan)



# -------------------------------------------------------
# 3. Recode education columns in a vectorized way
# -------------------------------------------------------
# educ_cols = ["EDUC99_MOM", "EDUC99_MOM2", "EDUC99_POP", "EDUC99_POP2", "EDUC99"]
# df[educ_cols] = df[educ_cols].apply(recode_educ_column)

# -------------------------------------------------------
# 4. Convert age into buckets (vectorized with pd.cut)
# -------------------------------------------------------
age_bins = [0, 18, 25, 35, 45, 55, 65, np.inf]
age_labels = ["under_18", "18–24", "25–34", "35–44", "45–54", "55–64", "65+"]
age_cols = ["AGE_MOM", "AGE_MOM2", "AGE_POP", "AGE_POP2", "AGE"]

for col in age_cols:
    new_col = f"{col}_bucket"
    df[new_col] = pd.cut(
        df[col],
        bins=age_bins,
        labels=age_labels,
        right=False
    ).astype("object")  # Ensure dtype is object to avoid mismatches

# -------------------------------------------------------
# 5. Extract First Two Valid Parent Age Buckets (Vectorized)
# -------------------------------------------------------
parent_age_bucket_cols = ["AGE_MOM_bucket", "AGE_MOM2_bucket", "AGE_POP_bucket", "AGE_POP2_bucket"]

# Extract the relevant columns as a NumPy array
parent_age_array = df[parent_age_bucket_cols].values

# Initialize arrays to hold the first two valid ages
parent1_age = np.full(len(df), np.nan, dtype=object)
parent2_age = np.full(len(df), np.nan, dtype=object)

# Vectorized extraction using NumPy
# Iterate over each row's parent ages
for i in range(len(df)):
    valid_ages = parent_age_array[i][~pd.isna(parent_age_array[i])]
    if len(valid_ages) > 0:
        parent1_age[i] = valid_ages[0]
    if len(valid_ages) > 1:
        parent2_age[i] = valid_ages[1]

# Assign to DataFrame with explicit dtype
df["PARENT1_AGE_BUCKET"] = parent1_age
df["PARENT2_AGE_BUCKET"] = parent2_age

# -------------------------------------------------------
# 6. Pick first two valid parent education codes (vectorized)
# -------------------------------------------------------
parent_educ_cols = ["EDUC99_MOM", "EDUC99_MOM2", "EDUC99_POP", "EDUC99_POP2"]

# Verify that all parent education columns exist
missing_parent_educ_cols = [col for col in parent_educ_cols if col not in df.columns]
if missing_parent_educ_cols:
    raise ValueError(f"The following parent education columns are missing from the DataFrame: {missing_parent_educ_cols}")

# Extract the relevant columns as a NumPy array
parent_educ_array = df[parent_educ_cols].values

# Initialize arrays to hold the first two valid education codes
parent1_educ = np.full(len(df), np.nan, dtype=float)
parent2_educ = np.full(len(df), np.nan, dtype=float)
for i in range(len(df)):
    # Extract non-NaN education codes
    valid_educs = parent_educ_array[i][~pd.isna(parent_educ_array[i])]
    if len(valid_educs) > 0:
        parent1_educ[i] = valid_educs[0]
    if len(valid_educs) > 1:
        parent2_educ[i] = valid_educs[1]

# Assign to DataFrame
df["PARENT1_EDUC"] = parent1_educ
df["PARENT2_EDUC"] = parent2_educ

# -------------------------------------------------------
# 7. Recode hours worked (replace 999/997 with mean)
# -------------------------------------------------------
invalid_hours = [999, 997]
df["UHRSWORKT"] = df["UHRSWORKT"].replace(invalid_hours, np.nan)
mean_hours = df["UHRSWORKT"].mean()
df["UHRSWORKT"].fillna(mean_hours, inplace=True)

# -------------------------------------------------------
# 8. Compute hourly wage
# -------------------------------------------------------
# To avoid division by zero, replace any zero UHRSWORKT with a small number or NaN
valid_mean = df.loc[df["INCWAGE"] != 99999999, "INCWAGE"].mean()
df["INCWAGE"] = df["INCWAGE"].replace(99999999, valid_mean)
df["UHRSWORKT"] = df["UHRSWORKT"].replace(0, np.nan)
df["hr_wage"] = df["INCWAGE"] / df["UHRSWORKT"]
df["hr_wage"].replace([np.inf, -np.inf], np.nan, inplace=True)  # Handle division by zero if any
df["hr_wage"].fillna(df["hr_wage"].mean(), inplace=True)  # Replace NaNs resulting from division by zero

# -------------------------------------------------------
# 9. Compute tenure
# -------------------------------------------------------
df["TENURE"] = df["AGE"] - df["EDUC99"] - 7

# -------------------------------------------------------
# 10. Define columns for your modeling
# -------------------------------------------------------
categorical_columns = [
    "SEX", "RACE", "YEAR",   "AGE_bucket",
    "VETSTAT", "PARENT1_AGE_BUCKET", "PARENT2_AGE_BUCKET"
]
numeric_columns = ["EDUC99", "PARENT1_EDUC", "PARENT2_EDUC", "TENURE"]

target_variable = "hr_wage"
weight_column = "ASECWTH"

# (Optional) Quick sanity check


In [ ]:
df['INCWAGE'].max()

99999999

In [30]:
df['EDUC99_MOM'].isna().all()

False

## Data-Exploration

In [31]:
create_report = False
if create_report:
    report = sv.analyze(df)
    report.show_html("sweetviz_report.html")


## Models

In [32]:
# Define X and y
# -------------------------------------------------------------------
# Prepare features (X) and target (y)
# -------------------------------------------------------------------
# 1. Separate numeric features
X_numeric = df[numeric_columns]

# 2. Convert categorical columns to string and handle NaNs by filling with 'Missing'
df_categorical_str = df[categorical_columns].astype(str).fillna('Missing')

# Additionally, replace 'nan' strings resulting from original NaNs with 'Missing'
df_categorical_str.replace('nan', 'Missing', inplace=True)

# 3. One-hot encode categorical features, including 'Missing' as a category
df_one_hot = pd.get_dummies(df_categorical_str, dummy_na=False, drop_first=True)

# 4. Concatenate numeric and one-hot-encoded features
X = pd.concat([X_numeric, df_one_hot], axis=1)

# 5. Define the target variable (y) and sample weights
y = df[target_variable]
sample_weight = df[weight_column]

# -------------------------------------------------------
# 12. Impute Missing Values in Numeric Features
# -------------------------------------------------------
# Identify numeric columns in X_numeric
from sklearn.impute import SimpleImputer

# Initialize SimpleImputer for numeric data
numeric_imputer = SimpleImputer(strategy='mean')

# Fit and transform numeric features
X_numeric_imputed = pd.DataFrame(
    numeric_imputer.fit_transform(X_numeric),
    columns=X_numeric.columns,
    index=X_numeric.index
)

# Update X_numeric with imputed values
X_numeric = X_numeric_imputed

# Replace the numeric part in X with imputed data
X.update(X_numeric)

# -------------------------------------------------------
# 13. Ensure No NaNs in One-Hot Encoded Features
# -------------------------------------------------------
# Since we filled NaNs in categorical features with 'Missing' before encoding,
# there should be no NaNs in df_one_hot. However, verify just in case.
assert not X.isnull().any().any(), "There are still NaNs in the feature set."

# -------------------------------------------------------
# 14. Handle Missing Values in Target Variable (y)
# -------------------------------------------------------
# Drop rows where target variable is NaN
valid_rows = y.notna()
X = X[valid_rows]
y = y[valid_rows]
sample_weight = sample_weight[valid_rows]

# -------------------------------------------------------
# 15. Final Sanity Checks
# -------------------------------------------------------
print("\nFinal data checks:")
print(f"Number of samples: {X.shape[0]}")
print(f"Number of features: {X.shape[1]}")
print(f"Any NaNs in features: {X.isnull().any().any()}")
print(f"Any NaNs in target: {y.isnull().any()}")



#########################################################################################################
##############################################################################################################################
##############################################################################################################################
#####Comparing Models
###############################################################
###############################################################


# -------------------------------------------------------
# 1. Prepare the Dataset (Assuming X and y are already defined)
# -------------------------------------------------------
# Ensure no NaNs before training
X = X.fillna(0)  # Replace NaNs with 0 (Alternative: Impute with mean)
y = y.fillna(y.mean())  # Impute target variable if needed

# Split dataset for fair comparison
X_train, X_test, y_train, y_test, sw_train, sw_test = train_test_split(
    X, y, sample_weight, test_size=0.2, random_state=42
)

# -------------------------------------------------------
# 2. Measure Initial Resource Usage (Linear Regression)
# -------------------------------------------------------
process = psutil.Process()
start_cpu_times = process.cpu_times()
mem_info_start = process.memory_info().rss  # Resident Set Size in bytes
start_time = time.time()

# -------------------------------------------------------
# 3. Fit the Linear Regression Model
# -------------------------------------------------------
lr_model = LinearRegression()
lr_model.fit(X_train, y_train, sample_weight=sw_train)
lr_r2 = lr_model.score(X_test, y_test, sample_weight=sw_test)

# -------------------------------------------------------
# 4. Measure Resource Usage After Training (Linear Regression)
# -------------------------------------------------------
end_time = time.time()
end_cpu_times = process.cpu_times()
mem_info_end = process.memory_info().rss

# Calculate differences
lr_run_time = end_time - start_time
lr_cpu_time_used = (
    (end_cpu_times.user + end_cpu_times.system)
    - (start_cpu_times.user + start_cpu_times.system)
)
lr_mem_used = (mem_info_end - mem_info_start) / (1024 * 1024)  # Convert bytes to MB

# -------------------------------------------------------
# 5. Print Linear Regression Results
# -------------------------------------------------------
print("\n===== Linear Regression Performance =====")
print(f"R² Score: {lr_r2:.4f}")
print(f"Time taken: {lr_run_time:.4f} seconds")
print(f"CPU time used: {lr_cpu_time_used:.4f} seconds (user + system)")
print(f"Approx. additional RAM used: {lr_mem_used:.4f} MB")

# -------------------------------------------------------
# 6. Define Optuna Objective Function for XGBoost
# -------------------------------------------------------
def objective(trial):
    """Hyperparameter tuning using Optuna"""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "random_state": 42
    }
    
    model = xgb.XGBRegressor(**params)
    model.fit(
        X_train, y_train,
        sample_weight=sw_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
    
    pred = model.predict(X_test)
    return r2_score(y_test, pred)

# -------------------------------------------------------
# 7. Measure Initial Resource Usage (XGBoost)
# -------------------------------------------------------
process = psutil.Process()
start_cpu_times = process.cpu_times()
mem_info_start = process.memory_info().rss  # Resident Set Size in bytes
start_time = time.time()

# -------------------------------------------------------
# 8. Run Optuna Hyperparameter Optimization
# -------------------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # Adjust trials as needed

# Best parameters
best_params = study.best_params
print("\nBest XGBoost Parameters:", best_params)

# -------------------------------------------------------
# 9. Train Final XGBoost Model with Best Hyperparameters
# -------------------------------------------------------
xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(X_train, y_train, sample_weight=sw_train)

# -------------------------------------------------------
# 10. Evaluate the Model (R² Score)
# -------------------------------------------------------
y_pred_xgb = xgb_model.predict(X_test)
xgb_r2 = r2_score(y_test, y_pred_xgb)

# -------------------------------------------------------
# 11. Measure Time and Resource Usage After Training (XGBoost)
# -------------------------------------------------------
end_time = time.time()
end_cpu_times = process.cpu_times()
mem_info_end = process.memory_info().rss

# Calculate differences
xgb_run_time = end_time - start_time
xgb_cpu_time_used = (
    (end_cpu_times.user + end_cpu_times.system)
    - (start_cpu_times.user + start_cpu_times.system)
)
xgb_mem_used = (mem_info_end - mem_info_start) / (1024 * 1024)  # Convert bytes to MB

# -------------------------------------------------------
# 12. Print XGBoost Results
# -------------------------------------------------------
print("\n===== XGBoost Performance =====")
print(f"R² Score: {xgb_r2:.4f}")
print(f"Time taken: {xgb_run_time:.4f} seconds")
print(f"CPU time used: {xgb_cpu_time_used:.4f} seconds (user + system)")
print(f"Approx. additional RAM used: {xgb_mem_used:.4f} MB")

# -------------------------------------------------------
# 13. Compare Linear Regression vs XGBoost
# -------------------------------------------------------
print("\n===== Model Comparison =====")
print(f"{'Metric':<25} {'Linear Regression':<20} {'XGBoost':<20}")
print("-" * 65)
print(f"{'R² Score':<25} {lr_r2:<20.4f} {xgb_r2:<20.4f}")
print(f"{'Time Taken (s)':<25} {lr_run_time:<20.4f} {xgb_run_time:<20.4f}")
print(f"{'CPU Time Used (s)':<25} {lr_cpu_time_used:<20.4f} {xgb_cpu_time_used:<20.4f}")
print(f"{'Memory Used (MB)':<25} {lr_mem_used:<20.4f} {xgb_mem_used:<20.4f}")

[I 2025-01-28 14:11:10,032] A new study created in memory with name: no-name-5791aeff-3307-4b4d-bc2c-93f0bb2a9351



Final data checks:
Number of samples: 100000
Number of features: 30
Any NaNs in features: False
Any NaNs in target: False

===== Linear Regression Performance =====
R² Score: 0.9874
Time taken: 0.1055 seconds
CPU time used: 0.7969 seconds (user + system)
Approx. additional RAM used: 0.0000 MB


[I 2025-01-28 14:11:11,909] Trial 0 finished with value: 0.9998267138129874 and parameters: {'n_estimators': 389, 'max_depth': 8, 'learning_rate': 0.18037347501244516, 'subsample': 0.5726551158693429, 'colsample_bytree': 0.5955087150902499}. Best is trial 0 with value: 0.9998267138129874.
[I 2025-01-28 14:11:12,429] Trial 1 finished with value: 0.9998580913201494 and parameters: {'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.06218866507451881, 'subsample': 0.6118279993874646, 'colsample_bytree': 0.8728507966376917}. Best is trial 1 with value: 0.9998580913201494.
[I 2025-01-28 14:11:13,689] Trial 2 finished with value: 0.9997207569180964 and parameters: {'n_estimators': 387, 'max_depth': 3, 'learning_rate': 0.013684393800767698, 'subsample': 0.9581641884965826, 'colsample_bytree': 0.5763714597170853}. Best is trial 1 with value: 0.9998580913201494.
[I 2025-01-28 14:11:15,271] Trial 3 finished with value: 0.9998658747582126 and parameters: {'n_estimators': 455, 'max_depth': 4,


Best XGBoost Parameters: {'n_estimators': 430, 'max_depth': 5, 'learning_rate': 0.032754277374995595, 'subsample': 0.6832492279017804, 'colsample_bytree': 0.7560385276911048}

===== XGBoost Performance =====
R² Score: 0.9999
Time taken: 32.9752 seconds
CPU time used: 267.1250 seconds (user + system)
Approx. additional RAM used: 40.5078 MB

===== Model Comparison =====
Metric                    Linear Regression    XGBoost             
-----------------------------------------------------------------
R² Score                  0.9874               0.9999              
Time Taken (s)            0.1055               32.9752             
CPU Time Used (s)         0.7969               267.1250            
Memory Used (MB)          0.0000               40.5078             


In [ ]:
X_train.head()

,EDUC99,PARENT1_EDUC,PARENT2_EDUC,TENURE,SEX_2,RACE_200,RACE_300,RACE_650,AGE_bucket_25–34,AGE_bucket_35–44,AGE_bucket_45–54,AGE_bucket_55–64,AGE_bucket_65+,AGE_bucket_<18,VETSTAT_1,VETSTAT_2,PARENT1_AGE_BUCKET_25–34,PARENT1_AGE_BUCKET_35–44,PARENT1_AGE_BUCKET_45–54,PARENT1_AGE_BUCKET_55–64,PARENT1_AGE_BUCKET_65+,PARENT1_AGE_BUCKET_<18,PARENT1_AGE_BUCKET_Missing,PARENT2_AGE_BUCKET_25–34,PARENT2_AGE_BUCKET_35–44,PARENT2_AGE_BUCKET_45–54,PARENT2_AGE_BUCKET_55–64,PARENT2_AGE_BUCKET_65+,PARENT2_AGE_BUCKET_<18,PARENT2_AGE_BUCKET_Missing
75220,6,6.000000,15.000000,2,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False
48955,10,11.020831,11.490234,64,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True
44966,11,11.020831,11.490234,27,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True
13568,15,10.000000,11.490234,0,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
92727,5,6.000000,11.490234,4,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True
